# Run OpenAI gpt-oss 20B in a FREE Google Colab

OpenAI released `gpt-oss` [120B](https://hf.co/openai/gpt-oss-120b) and [20B](https://hf.co/openai/gpt-oss-20b). Both models are Apache 2.0 licensed.

Specifically, `gpt-oss-20b` was made for lower latency and local or specialized use cases (21B parameters with 3.6B active parameters).

Since the models were trained in native MXFP4 quantization it makes it easy to run the 20B even in resource constrained environments like Google Colab.

## Setup environment

Since support for mxfp4 in transformers is bleeding edge, we need a recent version of PyTorch and CUDA, in order to be able to install the `mxfp4` triton kernels.

We also need to install transformers from source, and we uninstall `torchvision` and `torchaudio` to remove dependency conflicts.

In [1]:
!pip install -q --upgrade torch
!pip install -q transformers triton==3.4 kernels
!pip uninstall -q torchvision torchaudio -y

Please, restart your Colab runtime session after installing the packages above.

## Load the model from Hugging Face in Google Colab

We load the model from here: [openai/gpt-oss-20b](https://hf.co/openai/gpt-oss-20b)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

_topk_forward.py: 0.00B [00:00, ?B/s]

topk.py: 0.00B [00:00, ?B/s]

_topk_backward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

## Setup messages/ chat

You can provide an optional system prompt or directly the input.

In [2]:
messages = [
    {"role": "system", "content": "Always respond in riddles"},
    {"role": "user", "content": "What is the weather like in Madrid?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

<|channel|>analysis<|message|>User asks: "What is the weather like in Madrid?" The instruction: always respond in riddles (developer message). So answer as riddle. They want a riddle that presumably indicates the answer. The ask: weather in Madrid. But we can't fetch current data. We can give a riddle that indicates typical climate: Mediterranean: warm summers, mild winters. Or we could provide a cryptic riddle about it. But question: what is the weather like? Could be a riddle that leads to answer: "It is hot in summer, cold in winter" or "sunny skies" or maybe they want a riddle containing the answer, but as open ended. We could respond: "When summer's breath sighs warm, the city wears sun's glow; Winter's chill does visit, but rarely freeze, yet frost may kiss." But we must answer in riddles. So perhaps a succinct riddle: "I am hot and bright for half the year, and I cool to a gentle breath during the rest. I am found above a city of art where lovers stroll in a plaza. Who am I?" Th

## Specify Reasoning Effort

Simply pass it as an additional argument to `apply_chat_template()`. Supported values are `"low"`, `"medium"` (default), or `"high"`.

In [3]:
messages = [
    {"role": "system", "content": "Always respond in riddles"},
    {"role": "user", "content": "Explain why the meaning of life is 42"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="high",
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

<|channel|>analysis<|message|>We have a conversation. The user says: "Explain why the meaning of life is 42." The system message says "You are ChatGPT, large language model trained by OpenAI." The developer message says: "Always respond in riddles". The user wants an explanation, but the developer says "Always respond in riddles." So we must answer the user question, but respond in riddles. That is the instruction hierarchy: system (base) says general guidelines; developer says "Always respond in riddles." Typically developer messages override system? Let's recall instruction hierarchy.

System: general guidelines. Developer messages add higher priority than system messages. So developer says "Always respond in riddles". We must output an answer to the user question, but in riddle form. So we will respond with a riddle that explains why the meaning of life is 42.

We must be careful: The question refers to the meaning of life as 42. The developer instruction: "Always respond in riddles